In [1]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset
from scipy.special import softmax
import evaluate
import numpy as np

In [2]:
ds = load_dataset('imdb')

In [3]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [6]:
def preprocess_text(ds):
    return tokenizer(ds["text"], truncation=True)

In [7]:
tokenized_ds = ds.map(preprocess_text, batched=True)

In [8]:
roc_auc = evaluate.load('roc_auc', 'binary')

In [56]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=1)
    probabilities = softmax(logits)
    y_prob = probabilities.max(axis=1)
    return roc_auc.compute(prediction_scores=y_prob, references=labels)
    

In [57]:
small_train = tokenized_ds["train"].shuffle(seed=42).select(range(100))
small_eval = tokenized_ds["test"].shuffle(seed=42).select(range(100))

In [58]:
training_args = TrainingArguments(
    output_dir="sentiment",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

In [69]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

In [70]:
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc
1,0.027300,0.555814,0.646450
2,0.058300,0.417055,0.736487


TrainOutput(global_step=3126, training_loss=0.04669235794496933, metrics={'train_runtime': 811.5801, 'train_samples_per_second': 61.608, 'train_steps_per_second': 3.852, 'total_flos': 6556904415524352.0, 'train_loss': 0.04669235794496933, 'epoch': 2.0})

In [72]:
trainer.evaluate()

{'eval_loss': 0.41705480217933655,
 'eval_roc_auc': 0.7364868671999999,
 'eval_runtime': 105.1284,
 'eval_samples_per_second': 237.805,
 'eval_steps_per_second': 14.868,
 'epoch': 2.0}